In [16]:
import numpy as np
import itertools
from hmmlearn import hmm
from models_gaussian import StandardGaussianHMM, GaussianDenseHMM, HMMLoggingMonitor
import time
from tqdm import tqdm
from ssm.util import find_permutation
import pickle
import json
from pathlib import Path
import wandb
from utils import dtv, permute_embeddings, compute_stationary,  empirical_coocs
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.special import erf
from sklearn.cluster import KMeans

In [17]:
np.random.seed(2022)

simple_model = {"mu": 10,
                "sigma": 1}

complicated_model = {"mu": 5,
                     "sigma": 2}

data_sizes = [(100, 400, 4)]  # (s, T, n) TODO:  fill

ls = (2, 3, 4,  5)
ms = (4, 8, 12, 64, 128)

In [18]:
def prepare_params(n, simple_model=True):
    A = np.exp(np.random.uniform(0, 5, size=(n, n)))
    A /= A.sum(axis=1)[:, np.newaxis]

    pi = compute_stationary(A)

    if simple_model:
        mu = np.arange(n) * 10
        sigma = np.ones(shape=n)
    else:
        mu = np.random.uniform(0, n * 3, size=n)
        sigma = np.random.uniform(.5, 1.75, size=n)
    return pi, A, mu, sigma


def my_hmm_sampler(pi, A, mu, sigma, T):
    n = pi.shape[0]
    X = [np.random.choice(np.arange(n), 1, replace=True, p=pi)]
    for t in range(T - 1):
        X.append(np.random.choice(np.arange(n), 1, replace=True, p=A[X[t][0], :]))
    Y = np.concatenate([np.random.normal(mu[s[0]], sigma[s[0]], 1) for s in X]).reshape(-1, 1)
    return X, Y


def init_experiment(dsize, simple_model):
    s = dsize[0]
    T = dsize[1]
    n = dsize[2]
    pi, A, mu, sigma = prepare_params(n, simple_model)

    data = [my_hmm_sampler(pi, A, mu, sigma, T) for _ in range(s)]
    X_true = np.concatenate([np.concatenate(y[0]) for y in data])  # states
    Y_true = np.concatenate([x[1] for x in data])  # observations
    lengths = [len(x[1]) for x in data]

    return s, T, n, pi, A, mu, sigma, X_true, Y_true, lengths

In [19]:
def to_discrete_q(X, m):
    nodes = np.concatenate([np.quantile(X, [i / m for i in range(1, m)]), np.array([np.infty])])
    return (X > nodes.reshape(1, -1)).sum(axis=-1).reshape(-1, 1), nodes.reshape(-1)

def to_discrete(X, m):
    kmeans = KMeans(n_clusters=m, random_state=0).fit(Y_true)
    nodes_tmp = np.sort(kmeans.cluster_centers_, axis=0)
    nodes = np.concatenate([(nodes_tmp[1:] + nodes_tmp[:-1]) / 2, np.array([[np.infty]])])
    return (X > nodes.reshape(1, -1)).sum(axis=-1).reshape(-1, 1), nodes.reshape(-1)

def empirical_cooc_prob(Xd, m, lengths):
    freqs, gt_omega_emp = empirical_coocs(Xd, m, lengths=lengths)
    return np.reshape(gt_omega_emp, newshape=(m, m))

def normal_cooc_prob(means, covars, Qs, A):
    A_stationary = compute_stationary(A, False)
    B_scalars_tmp = .5 * (1 + erf((Qs[:-1, np.newaxis] - np.transpose(means)) / np.transpose(covars) / np.sqrt(2)))
    B_scalars_tmp = np.concatenate([np.zeros((1, B_scalars_tmp.shape[1])), B_scalars_tmp, np.ones((1, B_scalars_tmp.shape[1]))], axis=0)
    B_scalars = np.transpose(B_scalars_tmp[1:, :] - B_scalars_tmp[:-1, :])
    theta = A * A_stationary[:, None]
    return np.matmul(np.transpose(B_scalars), np.matmul(theta, B_scalars))

In [20]:
def visualize_distribution(mu, sigma, n, simple_model):
    x = np.linspace(min(mu) - 3 * max(sigma), max(mu) + 3 * max(sigma), 10000)
    for i in range(n):
        plt.plot(x, stats.norm.pdf(x, mu[i], sigma[i]), label=str(i))
    plt.title(f"Normal PDFs n={n}-s={s}-T={T}-simple={simple_model}")
    plt.show()

def visualize_matrix(mat, title="", vmax=1):
    sns.heatmap(mat, cmap="hot", vmax=vmax)
    plt.title(title)
    plt.show()

In [21]:
M=10
EM_ITER = 20
l = 8
ITER = 100000
TOLERANCE = 1e-4

def em_scheduler(max_lr, it):
    if it <= np.ceil(ITER / 3):
        return max_lr * np.cos(3 * (np.ceil(ITER / 3) - it) * np.pi * .33 / ITER)
    else:
        return max_lr * np.cos((it - np.ceil(ITER / 3)) * np.pi * .66 / ITER) ** 3

mstep_cofig = {"cooc_lr": 0.1, "cooc_epochs": ITER, "l_uz": 10,
               "em_lr":  0.01, "em_epochs":20, 'loss_type': 'square', "scheduler": em_scheduler}
s, T, n, pi, A, mu, sigma, X_true, Y_true, lengths = init_experiment(dsize=(100, 100, 10), simple_model=True)

In [22]:
t = time.localtime()

true_values = {
    "states": X_true,
    "transmat": A,
    "startprob": pi,
    "means": mu,
    "covars": sigma
}

wandb_params = {
    "init": {
        "project": "gaussian-dense-hmm",
        "entity": "cirglaboratory",
        "save_code": True,
        "group": f"benchmark-{t.tm_year}-{t.tm_mon}-{t.tm_mday}-debug4",
        "job_type": f"n={n}-s={s}-T={T}-simple={simple_model}",
        "name": f"PDFs",
        "reinit": True
    },
    "config": {
        "n": n,
        "s": s,
        "T": T,
        "model": None,
        "m": None,
        "l": None,
        "lr":  0,
        "em_epochs":  0,
        "em_iter": EM_ITER,
        "cooc_epochs": ITER,
        "epochs": 0,
        "simple_model": simple_model
    }
}

wandb.init(**wandb_params["init"], config=wandb_params["config"])

wandb_params["init"].update({"job_type": f"n={n}-s={s}-T={s}-simple={simple_model}",
                             "name": f"dense--l={l}-lr={mstep_cofig['cooc_lr']}-epochs={mstep_cofig['cooc_epochs']}-{time.asctime()}"})
wandb_params["config"].update(dict(model="dense_cooc_em_penalty", m=n, l=l, lr=mstep_cofig['cooc_lr'], em_epochs=10,
                                   em_iter=EM_ITER, cooc_epochs=mstep_cofig['cooc_epochs'], epochs=mstep_cofig['cooc_epochs'],
                                   loss_type='square', scheduler=True))

wandb.init(**wandb_params["init"], config=wandb_params["config"])


accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
covars_mae,▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▃▃▄▄▅▅▆▆▆▇▇▇██████████
means_mae,▁▁▂▂▂▁▁▂████████████████████████████████
omage_dtv,█▂▂▃▄▄▄▄▂▂▁▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅
startprob_dtv,▁▇▆▃▅▅▅▄▃▂▄▇████████████████████████████
time,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
total_log_prob,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
transmat_dtv,███▇▇▇▅▃▁▁▁▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄
accuracy,0.3017
covars_mae,1124.77778
means_mae,30.99093


In [32]:
nodes = np.concatenate([np.array([-np.infty, Y_true.min() +  1e-3]), nodes, np.array([Y_true.max() - 1e-3])])

In [33]:
for  _  in range(10):
    mstep_cofig = {"cooc_lr": 0.3, "cooc_epochs": ITER, "l_uz": 10,
               "em_lr":  0.01, "em_epochs":20, 'loss_type': 'square', "scheduler": em_scheduler}
    wandb_params["init"].update({"name": f"dense--l={l}-lr={mstep_cofig['cooc_lr']}-epochs={mstep_cofig['cooc_epochs']}-{time.asctime()}"})
    hmm_monitor = HMMLoggingMonitor(tol=TOLERANCE, n_iter=0, verbose=True,
                                    wandb_log=True, wandb_params=wandb_params, true_vals=true_values,
                                    log_config={'metrics_after_convergence': True})
    densehmm = GaussianDenseHMM(n, mstep_config=mstep_cofig,
                                covariance_type='diag', opt_schemes={"cooc", "em"},
                                nodes=nodes,
                                discrete_observables=n+2, em_iter=20,
                                logging_monitor=hmm_monitor,
                                init_params="", params="stmc", early_stopping=True)
    densehmm.means_ = mu.reshape(-1, 1)
    start = time.perf_counter()
    densehmm.fit_coocs(Y_true, lengths)
    # densehmm.fit(Y_true, lengths)
    time_tmp = time.perf_counter() - start

accuracy,▁
means_mae,▁
startprob_dtv,▁
time,▁
total_log_prob,▁
transmat_dtv,▁
accuracy,0.2684
covars_mae,nan
means_mae,31.21324
omage_dtv,nan
startprob_dtv,0.06173


ValueError: Dimensions must be equal, but are 12 and 15 for '{{node sub_6}} = Sub[T=DT_DOUBLE](Placeholder, MatMul_1)' with input shapes: [12,12], [15,15].

In [ ]:
feed_dict = {densehmm.omega_gt_ph: omega_emp}
feed_dict[densehmm.A_stationary] = compute_stationary(densehmm.session.run(densehmm.A_from_reps_hmmlearn), verbose=False)
feed_dict[densehmm.lr_cooc_placeholder] = 0.1
omega_learned = densehmm.session.run(densehmm.omega, feed_dict)

In [ ]:
vmax = max(omega_learned.max(), omega_gt.max(), (omega_learned - omega_gt).max()) +  1e-1
visualize_matrix(omega_learned, title="Learned coocurences", vmax=vmax)
visualize_matrix(omega_gt, title="GT distribution coocurences",  vmax=vmax)
visualize_matrix(omega_emp, title="Empirical coocurences", vmax=vmax)

In [ ]:
plt.vlines(nodes, np.zeros(nodes.shape), np.ones(nodes.shape) * 0.33, color="blue", alpha = 0.8)
plt.hist(Y_true, bins=[i for i in range(100)], color="black", density=True)
x = np.linspace(min(mu) - 3 * max(sigma), max(mu) + 3*max(sigma), 10000).reshape(-1)
for i in range(n):
    plt.plot(x, stats.norm.pdf(x, densehmm.means_[i], densehmm.covars_[i, 0]).reshape(-1), label=str(i), color="red")
plt.title("Distributions and nodes")
plt.show()

In [ ]:
stats.norm.pdf(x, densehmm.means_[i], densehmm.covars_[i, 0]).reshape(-1)

In [ ]:
densehmm.fit(Y_true, lengths)

In [ ]:
densehmm.means_

In [ ]:
plt.vlines(nodes, np.zeros(nodes.shape), np.ones(nodes.shape) * 0.33, color="blue", alpha = 0.8)
plt.hist(Y_true, bins=[i for i in range(100)], color="black", density=True)
x = np.linspace(min(mu) - 3 * max(sigma), max(mu) + 3*max(sigma), 10000).reshape(-1)
for i in range(n):
    plt.plot(x, stats.norm.pdf(x, densehmm.means_[i], densehmm.covars_[i, 0]).reshape(-1), label=str(i), color="red")
plt.title("Distributions and nodes")
plt.show()